# SerpAPI Tool Helper

This notebook provides a quick rundown of how we construct an async SerpAPI tool. Let's start by query SerpAPI synchronously via their Python SDK:

In [2]:
from serpapi import GoogleSearch
from getpass import getpass

SERPAPI_API_KEY = getpass("Enter your SerpAPI API key: ")

params = {
    "api_key": SERPAPI_API_KEY,
    "engine": "google",
    "q": "latest news in the world",
}

search = GoogleSearch(params)
results = search.get_dict()

print(results)

{'search_metadata': {'id': '68bf117ac4f20fc5e44a90df', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/4712b6f99910a564/68bf117ac4f20fc5e44a90df.json', 'pixel_position_endpoint': 'https://serpapi.com/searches/4712b6f99910a564/68bf117ac4f20fc5e44a90df.json_with_pixel_position', 'created_at': '2025-09-08 17:25:14 UTC', 'processed_at': '2025-09-08 17:25:14 UTC', 'google_url': 'https://www.google.com/search?q=latest+news+in+the+world&oq=latest+news+in+the+world&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/4712b6f99910a564/68bf117ac4f20fc5e44a90df.html', 'total_time_taken': 1.49}, 'search_parameters': {'engine': 'google', 'q': 'latest news in the world', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'latest news in the world', 'total_results': 4480000000, 'time_taken_displayed': 0.38, 'organic_results_state': 'Results for exact spelling'}, 'inline_videos': [{'position': 1, 'title': 'ABC World N

Our results are provided in the `"organic_results"` key:

In [3]:
results["organic_results"]

[{'position': 1,
  'title': 'World | Latest News & Updates',
  'link': 'https://www.bbc.com/news/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world&ved=2ahUKEwjspfjD18mPAxV4STABHVmxBs8QFnoECCMQAQ',
  'displayed_link': 'https://www.bbc.com › news › world',
  'favicon': 'https://serpapi.com/searches/68bf117ac4f20fc5e44a90df/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': 'World · Six killed by Palestinian gunmen at Jerusalem bus stop · French government on the brink as prime minister faces defeat in confidence vote · Lone ...',
  'snippet_highlighted_words': ['Six killed by Palestinian gunmen at Jerusalem bus stop'],
  'sitelinks': {'inline': [{'title': 'BBC World',
     'link': 'https://www.bbc.com/news/world_radio_and_tv'},
    {'title': 'Middle East',
     'link': 'https://www.bbc.com/news/world/middle_east'},
    {'title': 'Africa', 'link': 'https://www.bbc.com/news/wo

We reformat this to extract only the most relevant information, such as the the title, source, link, and snippet. Let's use pydantic `BaseModel` to define this structure.

In [4]:
from pydantic import BaseModel

class Article(BaseModel):
    title: str
    source: str
    link: str
    snippet: str

    @classmethod
    def from_serpapi_result(cls, result: dict) -> "Article":
        return cls(
            title=result["title"],
            source=result["source"],
            link=result["link"],
            snippet=result["snippet"],
        )

In [6]:
articles = [Article.from_serpapi_result(result) for result in results["organic_results"]]
articles[0:3]

[Article(title='World | Latest News & Updates', source='BBC', link='https://www.bbc.com/news/world', snippet='World · Six killed by Palestinian gunmen at Jerusalem bus stop · French government on the brink as prime minister faces defeat in confidence vote · Lone ...'),
 Article(title='World news - breaking news, video, headlines and opinion', source='CNN', link='https://www.cnn.com/world', snippet='World · Police shoot dead fugitive father and find his three children missing for years in wilderness · Train collides with bus in Mexico, killing at least 8 ...'),
 Article(title='Breaking News, World News and Video from Al Jazeera', source='Al Jazeera', link='https://www.aljazeera.com/', snippet='Al Jazeera · Alcaraz dethrones Sinner to win US Open in four sets · Spain run riot in Turkiye as Merino scores hat-trick in 6-0 victory · F1: ...')]

## Using Async

All of this works, but it unfortunately is not async and the SerpAPI SDK does not support async either, so we much query the API directly using the `aiohttp` library.

In [7]:
import aiohttp

async with aiohttp.ClientSession() as session:
    async with session.get(
        "https://serpapi.com/search",
        params=params
    ) as response:
        results = await response.json()

results["organic_results"]

[{'position': 1,
  'title': 'World | Latest News & Updates',
  'link': 'https://www.bbc.com/news/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world&ved=2ahUKEwjspfjD18mPAxV4STABHVmxBs8QFnoECCMQAQ',
  'displayed_link': 'https://www.bbc.com › news › world',
  'favicon': 'https://serpapi.com/searches/68bf117ac4f20fc5e44a90df/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': 'World · Six killed by Palestinian gunmen at Jerusalem bus stop · French government on the brink as prime minister faces defeat in confidence vote · Lone ...',
  'snippet_highlighted_words': ['Six killed by Palestinian gunmen at Jerusalem bus stop'],
  'sitelinks': {'inline': [{'title': 'BBC World',
     'link': 'https://www.bbc.com/news/world_radio_and_tv'},
    {'title': 'Middle East',
     'link': 'https://www.bbc.com/news/world/middle_east'},
    {'title': 'Africa', 'link': 'https://www.bbc.com/news/wo

With that, we have all we need to build a fully async serpapi tool.

In [8]:
from langchain_core.tools import tool

@tool
async def serpapi(query: str) -> list[Article]:
    """Use this tool to search the web."""
    params = {
        "api_key": SERPAPI_API_KEY,
        "engine": "google",
        "q": query,
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(
            "https://serpapi.com/search",
            params=params
        ) as response:
            results = await response.json()
    return [Article.from_serpapi_result(result) for result in results["organic_results"]]

Note that because this tool is async, we cannot use `tool.func` to call it as before:

In [9]:
serpapi.func

Instead, we use `tool.coroutine`:

In [10]:
serpapi.coroutine

<function __main__.serpapi(query: str) -> list[__main__.Article]>

---